Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

4. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [3]:
failure = Node('failure', path_cost=math.inf) 
cutoff  = Node('cutoff',  path_cost=math.inf) 
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
FIFOQueue = deque
LIFOQueue = list


class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] 
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
# ==== Q1.a: Maze model ====
MAZE_ROWS    = 16
MAZE_COLUMNS = 24
ACTIONS      = {'UP', 'LEFT', 'RIGHT', 'DOWN'}
MOVE_DELTA = {
    'UP':    ( 1,  0),
    'DOWN':  (-1,  0),
    'RIGHT': ( 0,  1),
    'LEFT':  ( 0, -1),
}


def in_bounds(r, c):
    return 0 <= r < MAZE_ROWS and 0 <= c < MAZE_COLUMNS


class Maze(Problem):
    def __init__(self, initial=None, goal=None, walls=None, heuristic='h1'):
        super().__init__(initial=initial, goal=goal)
        self.walls = set() if walls is None else set(walls)
        self.heuristic = heuristic  

    def h1_weighted_manhattan(self, node):
        r, c   = node.state
        rg, cg = self.goal
        return abs(r - rg) * 1 + abs(c - cg) * 10

    def h2_weighted_chebyshev(self, node):
        r, c   = node.state
        rg, cg = self.goal
        return max(abs(r - rg) * 1, abs(c - cg) * 10)

    def h(self, node):
        if self.heuristic == 'h2':
            return self.h2_weighted_chebyshev(node)
    
        return self.h1_weighted_manhattan(node)

    # ---------- Transition model ----------
    def action_cost(self, s, a, s1):
        return 10 if a in ('LEFT', 'RIGHT') else 1

    def result(self, state, action):
        dr, dc = MOVE_DELTA[action]
        r, c = state
        return (r + dr, c + dc)

    def actions(self, state):
        r, c = state
        possible = set()
        for a in ACTIONS:
            dr, dc = MOVE_DELTA[a]
            nr, nc = r + dr, c + dc
            if in_bounds(nr, nc) and (nr, nc) not in self.walls:
                possible.add(a)
        return possible


### Testing the model

In [6]:
#Testing Maze class
S = (8, 10)
G = (11, 8)
WALLS = set()
WALLS |= {(r, 9) for r in range(6, 15)}        
WALLS |= {(r, 10) for r in range(10, 14)}      
WALLS |= {(r, c) for r in range(10, 12) for c in range(12, 14)}
m1 = Maze(initial=S, goal=G, walls=WALLS, heuristic='h1')

# Testing
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}
print(m1.result((4,12),'UP'))
print(m1.action_cost((4,12),'UP',(5,12)))
print(m1.actions((4,12)))


(5, 12)
1
{'LEFT', 'UP', 'RIGHT', 'DOWN'}


## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [7]:
#Your code for breadth-first search
from dataclasses import dataclass
from collections import deque
import math

@dataclass
class Stats:
    expanded: int
    max_frontier: int
    path_cost: float
    path: list

def BFS(problem):
    root = Node(problem.initial)
    if problem.is_goal(root.state):
        return Stats(0, 1, 0, [root.state])
    frontier = deque([root])
    reached = {root.state: 0}
    expanded = 0
    max_frontier = 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        node = frontier.popleft()
        expanded += 1
        for child in expand(problem, node):
            if child.state not in reached:
                if problem.is_goal(child.state):
                    return Stats(expanded, max_frontier, child.path_cost, path_states(child))
                reached[child.state] = child.path_cost
                frontier.append(child)
    return Stats(expanded, max_frontier, math.inf, [])

bfs_stats = BFS(m1)
print("BFS  →  expanded:", bfs_stats.expanded,
      " max_frontier:", bfs_stats.max_frontier,
      " path_cost:", bfs_stats.path_cost,
      " path_len:", len(bfs_stats.path)-1)


BFS  →  expanded: 129  max_frontier: 30  path_cost: 29  path_len: 11


In [8]:
#Your code for depth-first search with cycle-check
import math

def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)


def DFS_cycle(problem, limit=10):
    root = Node(problem.initial)
    frontier = [root]                    
    expanded = 0
    max_frontier = 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        node = frontier.pop()
        expanded += 1
        if problem.is_goal(node.state):
            return Stats(expanded, max_frontier, node.path_cost, path_states(node))
        elif len(path_states(node)) > limit:
            pass
        elif not is_cycle(node):
            for child in expand(problem, node):
                    frontier.append(child)
    return Stats(expanded, max_frontier, math.inf, [])


dfs_stats = DFS_cycle(m1, 15)
print("DFS  →  expanded:", dfs_stats.expanded,
      " max_frontier:", dfs_stats.max_frontier,
      " path_cost:", dfs_stats.path_cost,
      " path_len:", len(dfs_stats.path)-1 if dfs_stats.path else None)


DFS  →  expanded: 41447  max_frontier: 39  path_cost: 51  path_len: 15


In [9]:
#Your code for  iterative Deepening search with cycle-check
import math

def IDS(problem):
    prev_stats = None
    for limit in range(1, sys.maxsize):
        stats = DFS_cycle(problem, limit)
        if not prev_stats:
            prev_stats = stats
        else:
            prev_stats.expanded += stats.expanded
            prev_stats.max_frontier = max(prev_stats.max_frontier, stats.max_frontier)
        if stats.path_cost != math.inf:
            stats.expanded = prev_stats.expanded
            stats.max_frontier = prev_stats.max_frontier
            return stats


ids_stats = IDS(m1)
print("IDS  →  expanded:", ids_stats.expanded,
      " max_frontier:", ids_stats.max_frontier,
      " path_cost:", ids_stats.path_cost,
      " path_len:", len(ids_stats.path)-1 if ids_stats.path else None)


IDS  →  expanded: 112977  max_frontier: 31  path_cost: 29  path_len: 11


In [10]:
#Your code for uniform-cost search
def best_first_search(problem, f):
    root = Node(problem.initial)
    frontier = PriorityQueue([root], key=f)
    reached = {root.state: root}
    expanded = 0
    max_frontier = 1
    while frontier:
        max_frontier = max(max_frontier, len(frontier))
        node = frontier.pop()
        if problem.is_goal(node.state):
            return Stats(expanded, max_frontier, node.path_cost, path_states(node))
        expanded += 1
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return Stats(expanded, max_frontier, math.inf, [])


def UCS(problem):
    return best_first_search(problem, lambda n: n.path_cost)


ucs_stats = UCS(m1)
print("UCS  →  expanded:", ucs_stats.expanded,
      " max_frontier:", ucs_stats.max_frontier,
      " path_cost:", ucs_stats.path_cost,
      " path_len:", len(ucs_stats.path)-1 if ucs_stats.path else None)


UCS  →  expanded: 59  max_frontier: 27  path_cost: 29  path_len: 11


In [11]:
#Your code for A* Search
def A_star(problem, h):
    return best_first_search(problem, lambda n: n.path_cost + h(n))


astar_h1 = A_star(m1, m1.h1_weighted_manhattan)
astar_h2 = A_star(m1, m1.h2_weighted_chebyshev)
print("A* (h1 weighted Manhattan)  →  expanded:", astar_h1.expanded,
      " max_frontier:", astar_h1.max_frontier, " path_cost:", astar_h1.path_cost)
print("A* (h2 weighted Chebyshev)  →  expanded:", astar_h2.expanded,
      " max_frontier:", astar_h2.max_frontier, " path_cost:", astar_h2.path_cost)

A* (h1 weighted Manhattan)  →  expanded: 12  max_frontier: 15  path_cost: 29
A* (h2 weighted Chebyshev)  →  expanded: 22  max_frontier: 22  path_cost: 29


In [12]:
#Your code for greedy-best first search
def GBFS(problem, h):
    return best_first_search(problem, h)


gbfs_h1 = GBFS(m1, m1.h1_weighted_manhattan)
gbfs_h2 = GBFS(m1, m1.h2_weighted_chebyshev)
print("GBFS (h1 weighted Manhattan) → expanded:", gbfs_h1.expanded,
      " max_frontier:", gbfs_h1.max_frontier, " path_cost:", gbfs_h1.path_cost)
print("GBFS (h2 weighted Chebyshev) → expanded:", gbfs_h2.expanded,
      " max_frontier:", gbfs_h2.max_frontier, " path_cost:", gbfs_h2.path_cost)

GBFS (h1 weighted Manhattan) → expanded: 12  max_frontier: 15  path_cost: 29
GBFS (h2 weighted Chebyshev) → expanded: 12  max_frontier: 15  path_cost: 29


2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost|
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search |129|                      |30                      |29
| DFS with cycle check |41447                       |39                     |51
| IDS with cycle check |112977                      |31                      |29
| UCS                  |59                        |27                      |29
| A* Search            |12 (h1) / 22 (h2)         |15 (h1) / 22 (h2)       |29/29
| GBFS                 |12 (h1) / 12 (h2)         |15 (h1) / 15 (h2)       |29/29

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.



In both A* Search and GBFS, h1 weighted Manhattan shows less expansions, less maximum frontier sizes and achieve same correctness regarding the target 'path cost' compared with h2 weighted Chebyshev.

In conclusion, the best heuristic function is h1 weighted Manhattan.
